# Score Matrix

Use model to generate confusion matrix



In [ ]:
#import fnmatch
#import os
#import pathlib
#import random
#import time

import intel_extension_for_pytorch as ipex
import matplotlib.pyplot as plt
import numpy as np
#import psutil
import seaborn as sns
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
#from tqdm import tqdm
from PIL import Image
from data_loader import (
    TRAIN_DIR,
    VALID_DIR,
    augment_and_save,
    data_distribution,
    imagenet_stats,
    img_transforms,
    plot_data_distribution,
    show_data,
)
sns.set_theme(style="whitegrid", palette="husl")

# Specify Model to test

In [ ]:
# simple = False
# epochs = 12
# ipx = True 
# dropout = .5
# batch_size = 128
# # bc_resnet18_simpleFalse_IPEXTrue_Epochs12_dropout0.4_batch128
# model_name = "ON_resnet18_simpleFalse_IPEXTrue_Epochs40_dropout0.33_batch128"
# #resnet18_simpleFalse_IPEXTrue_Epochs12_dropout0.4_batch128
# #model_name = f"resnet18_simple{simple}_IPEX{ipx}_Epochs{epochs}_dropout{dropout}_batch{batch_size}"
# print(model_name)
# model_read = torch.jit.load(f"models/{model_name}.pt")

# model_read.eval()

# Define Predict Function

In [ ]:
def predict(InFldr, ScoreDict, model):
    import torchvision.transforms as transforms
    import torch.nn as nn
    import glob 
    model.eval()

    scores = []
    fns = []
    transform = transforms.Compose([
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize(*imagenet_stats),
    ])
    
    for fn in glob.glob(TEST_DIR+'*.png'):
        img = Image.open(fn)
        img_tensor = transform(img).reshape(1, 3, 1024, 1024)
        score = 0
        modelCPU = model.to("cpu")
        score = np.argmax(torch.softmax( modelCPU(img_tensor).detach(), dim =1).numpy().squeeze())
        fns.append(fn)
        scores.append(ScoreDict[score])
    return (scores, fns)
# model.eval()
# with torch.no_grad():
#     model_scripted = torch.jit.script(model.to("cpu"))  # Export to TorchScript
#     model_scripted.save(model_save_path)  # Jit Save

In [ ]:
#model_save_path
model_name = f"models/model_acc_84_device_xpu_lr_0.000214_epochs_20_jit.pt"
print(model_name)

model_read = torch.jit.load(model_name)
#model_read.eval()

# Scoring individual images with model
### Known Fire

In [ ]:
ytrue = []
ypred = []
TEST_DIR =  'data/colorEnhanced/val/Fire/'
scoreDict = {0:'Fire', 1:'NoFire'}
scores, fns = predict(TEST_DIR, scoreDict, model_read)
for score, fn in zip(scores, fns):
    print(f"{score}\t{fn}")
    if score == 'Fire': 
        ypred.append(1)
    else:
        ypred.append(0)
    ytrue.append(1)

# Scoring individual images with model
### Known NoFire

# Define functions and normalization parameters

In [ ]:
TEST_DIR =  'data/colorEnhanced/val/NoFire/'
scoreDict = {0:'Fire', 1:'NoFire'}
scores, fns = predict(TEST_DIR, scoreDict, model_read)
for score, fn in zip(scores, fns):
    print(f"{score}\t{fn}")
    if score == 'Fire': 
        ypred.append(1)
    else:
        ypred.append(0)
    ytrue.append(0)

In [ ]:
%matplotlib inline
fireDict = {0:'Fire', 1:'NoFire'}
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
YTRUE = [fireDict[val] for val in ytrue]
YPRED = [fireDict[val] for val in ypred]
labels=['Fire', 'NoFire']
cm = confusion_matrix(YTRUE, YPRED,  labels=labels)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()

plt.show()

In [ ]:
import pandas as pd
import numpy as np

usgs = pd.read_csv('data/USGS_NAIP_Sampled_Mapping.csv')
usgs['Longitude'] = usgs['Centroid'].apply(lambda x: float(x.split('(')[-1].replace(')','').replace(' ','').split(',')[0]))
usgs['Lattitude'] = usgs['Centroid'].apply(lambda x: float(x.split('(')[-1].replace(')','').replace(' ','').split(',')[1]))

ModisOutlineParadise = np.load('data/ModisOutlineParadise.npy')
ModisOutlineNorth = np.load('data/ModisOutlineNorth.npy')
ModisOutlineWest = np.load('data/ModisOutlineWest.npy' )

In [ ]:
def point_in_polygon(point, polygon):
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def point_out_polygon(point, polygon):
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return not inside

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from shapely.geometry import Polygon as GPoly

N_rnd = 600
np.random.seed(42)

polyP = Polygon(ModisOutlineParadise, closed=True,  fc=(1,0,0,0.3),  ec=(0,0,1,0.3))
polyW = Polygon(ModisOutlineWest,     closed=True,  fc=(1,0,0,0.3), ec=(0,0,1,0.3))
polyN = Polygon(ModisOutlineNorth,    closed=True,  fc=(1,0,0,0.3), ec=(0,0,1,0.3))

fig, ax = plt.subplots()

ax.add_patch(polyP)
ax.add_patch(polyN)
ax.add_patch(polyW)

# Plot the line segments joining the vertices

xp, yp = zip(*ModisOutlineParadise)
xw, yw = zip(*ModisOutlineWest)
xn, yn = zip(*ModisOutlineNorth)
xmin = min(min(xp),min(xw),min(xn))
xmax = max(max(xp),max(xw),max(xn))

ymin = min(min(yp),min(yw),min(yn))
ymax = max(max(yp),max(yw),max(yn))

xrnd = 4*np.random.sample(N_rnd) - 124
yrnd = 2*np.random.sample(N_rnd) + 39

# Color all points blue at first
Modis = np.vstack([ModisOutlineWest, ModisOutlineNorth, ModisOutlineParadise])
polyWG = GPoly(Modis)
prnd = np.array(list(zip(xrnd, yrnd)))
plt.scatter(xrnd, yrnd, s= 1, c='b')

# Color ModisOutlineWest points red
test = []
for point in prnd:
    test.append(point_in_polygon(point, ModisOutlineWest))
test = np.array(test)
plt.scatter(xrnd[test==True], yrnd[test==True], s= 1, c='r')

# Color ModisOutlineNorth points red
test = []
for point in prnd:
    test.append(point_in_polygon(point, ModisOutlineNorth))
test = np.array(test)
plt.scatter(xrnd[test==True], yrnd[test==True], s= 1, c='r')

# Color ModisOutlineParadise points red
test = []
for point in prnd:
    test.append(point_in_polygon(point, ModisOutlineParadise))
test = np.array(test)
plt.scatter(xrnd[test==True], yrnd[test==True], s= 1, c='r')


# Set the aspect ratio and limits
plt.gca().set_aspect('equal')
plt.xlim(-124, -120)
plt.ylim(39.0, 41.0)
plt.grid()
# Set labels and title
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('MODIS Regions Ideal Sampling')
#s = [1+10*i for i in range(len(x))]

# Show the plot
plt.show()


In [ ]:
import glob
SyntheticFire = []
SyntheticNoFire = []

path = 'data/synthetic/train/Fire/*.png'
for fn in glob.glob(path):
    SyntheticFire.append(fn.split('/')[-1])
                         
path = 'data/synthetic/val/Fire/*.png'
for fn in glob.glob(path):
    SyntheticFire.append(fn.split('/')[-1])
    
path = 'data/synthetic/train/NoFire/*.png'
for fn in glob.glob(path):
    SyntheticNoFire.append(fn.split('/')[-1])
                         
path = 'data/synthetic/val/NoFire/*.png'
for fn in glob.glob(path):
    SyntheticNoFire.append(fn.split('/')[-1])
    
len(SyntheticNoFire)

# Synthesize and MODIS Burn Area

sample N coordinates within and without the polygon

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from shapely.geometry import Polygon as GPoly
import random

use = len(SyntheticNoFire)   # min(VertFire.shape[0], VertNoFire.shape[0] )
print(use)


fig, ax = plt.subplots()

# polyP = Polygon(ModisOutlineParadise, closed=True,  fc=(1,0,0,0.3),  ec=(0,0,1,0.3))
# polyW = Polygon(ModisOutlineWest,     closed=True,  fc=(1,0,0,0.3), ec=(0,0,1,0.3))
# polyN = Polygon(ModisOutlineNorth,    closed=True,  fc=(1,0,0,0.3), ec=(0,0,1,0.3))

# ax.add_patch(polyP)
# ax.add_patch(polyN)
# ax.add_patch(polyW)

# Modis = np.vstack([ModisOutlineWest, ModisOutlineNorth, ModisOutlineParadise])
# polyWG = GPoly(Modis)
xmin = min(min(xp),min(xw),min(xn))
xmax = max(max(xp),max(xw),max(xn))

ymin = min(min(yp),min(yw),min(yn))
ymax = max(max(yp),max(yw),max(yn))

N_rnd = 2000

xrnd = 4*np.random.sample(N_rnd) - 124
yrnd = 2*np.random.sample(N_rnd) + 39
prnd = np.array(list(zip(xrnd, yrnd)))

# Create star shape for synthetic MODIS
import numpy as np
pi = np.pi
x = []
y = []
for k in range(0,5):
    r = 4/9
    x.append( r*np.cos(2*pi*k/5 + pi/2) - 122 )
    y.append( r*np.sin(2*pi*k/5 + pi/2) + 40 )
    r = 1
    x.append(  r*np.cos(2*pi*k/5 + pi/2+2*pi/5) - 122)
    y.append(  r*np.sin(2*pi*k/5 + pi/2+2*pi/5) + 40)
    
polyStar =  np.array(list(zip(x,y))) 
polyStarP = Polygon(polyStar, closed=True,  fc=(1,0,0,0.3),  ec=(0,0,1,0.3))

ax.add_patch(polyStarP)


# Color polyStar points red
test = []
for point in prnd:
    test.append(point_out_polygon(point, polyStar))
test = np.array(test)
VertNoFire = np.array(list(zip(xrnd[test==True], yrnd[test==True])))



# Color polyStar points red
test = []
for point in prnd:
    test.append(point_in_polygon(point, polyStar))
test = np.array(test)
VertFire = np.array(list(zip(xrnd[test==True], yrnd[test==True])))



plt.scatter(VertFire[:use,0], VertFire[:use,1], s= 1, c='r')
plt.scatter(VertNoFire[:use,0], VertNoFire[:use,1], s= 1, c='b')

# Set the aspect ratio and limits
plt.gca().set_aspect('equal')
plt.xlim(-124, -120)
plt.ylim(39.0, 41.0)
plt.grid()
# Set labels and title
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('MODIS Regions Ideal Sampling')

# Show the plot
plt.show()    


# Coorelate to USGS spreadsheet

In [ ]:
# TEST_DIR =  'data/colorEnhanced/val/NoFire/'
# scoreDict = {0:'Fire', 1:'NoFire'}
# scores, fns = predict(TEST_DIR, scoreDict, model_read)
# for score, fn in zip(scores, fns):
#     fnLookup = "_".join(fn.split('/')[4].split('_')[:6]) 
#     x = usgs[usgs.Name == fnLookup]['Longitude']+random.random()*.2
#     y = usgs[usgs.Name == fnLookup]['Lattitude']+random.random()*.2
#     #print(fn)
#     if score == 'Fire': 
#         ypred.append(1)
#         plt.scatter(x, y, s= 1, c='r')
#     else:
#         ypred.append(0)
#         plt.scatter(x, y, s= 1, c='b')
#     ytrue.append(0)

# TEST_DIR =  'data/colorEnhanced/val/Fire/'
# scoreDict = {0:'Fire', 1:'NoFire'}
# scores, fns = predict(TEST_DIR, scoreDict, model_read)
# for score, fn in zip(scores, fns):
#     fnLookup = "_".join(fn.split('/')[4].split('_')[:6]) 
#     x = usgs[usgs.Name == fnLookup]['Longitude']+random.random()*.2
#     y = usgs[usgs.Name == fnLookup]['Lattitude']+random.random()*.2
#     #print(fn)
#     if score == 'Fire': 
#         ypred.append(1)
#         plt.scatter(x, y, s= 1, c='r')
#     else:
#         ypred.append(0)
#         plt.scatter(x, y, s= 1, c='b')
#     ytrue.append(0)

## Notices and Disclaimers

Intel technologies may require enabled hardware, software or service activation.

No product or component can be absolutely secure. 

Your costs and results may vary. 

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. Other names and brands may be claimed as the property of others. 

In [ ]:
usgs[usgs['Name'] == 'm_4012032_sw_10_h_20160719']['Centroid']